In [ ]:
import pandas as pd

ci_filepath = r"https://raw.githubusercontent.com/data-to-insight/ERN-sessions/main/data/ChildIdentifiers.csv"

cc_filepath = r"https://raw.githubusercontent.com/data-to-insight/ERN-sessions/main/data/ChildCharacteristics.csv"

identifiers = pd.read_csv(ci_filepath)

characteristics = pd.read_csv(cc_filepath)

In [ ]:
# NaN = Not a Number
# NaT = Not a Time
# Na/pd = Not a Pandas value

#strftime format yyyy-MM-dd

#insert a new column formatting columns as a date (as dates are otherwise stored as strings if we examine identifiers.info)
identifiers['PersonBirthDate_dt'] = pd.to_datetime(identifiers['PersonBirthDate'], format="%Y-%m-%d")
identifiers['ExpectedPersonBirthdate_dt'] = pd.to_datetime(identifiers['PersonBirthDate'], format="%Y-%m-%d")


identifiers['PersonDeathDate_dt'] = pd.to_datetime(identifiers['PersonDeathDate'], format="%Y-%m-%d", errors="coerce") #errors="coerce" forces the errors to become 'NaT' so that we can still run calculations


print(identifiers.info())

In [ ]:
from dateutil.relativedelta import relativedelta

identifiers = pd.DataFrame(identifiers)

identifiers['Age'] = identifiers['PersonBirthDate_dt'].apply(lambda row: relativedelta(pd.to_datetime('today'), row).years)

identifiers


In [ ]:
from dateutil.relativedelta import relativedelta

# variables like pd.to_datetime is a function, and is snake_case.
# classes like pd.DataFrame is a class, and is CamelCase.

# a dictionary of lists
child_info = pd.DataFrame({'ChildId':['id1', 'id2', 'id3', 'id4', 'id5'],
							'Age at contact':[6, 12, 11, 1, 19],
							'SEX':['M', 'm', 'F', '', 'F'],
							'Birthday':['01/01/2002', '02/02/2003', '01/01/2000', '03/03/2023', '06/01/2012'],
							'CP Plan?':['Y', 'n', 'N', 'No', 'yES'],})

nhs_numbers = pd.DataFrame([{'ChildId':'id1',
							'NHS Number':'303'},
							{'ChildId':'id2',
							'NHS Number':'3u5029'},
							{'ChildId':'id3',
							'NHS Number': 'gqw3',},
							{'ChildId':'id4',
							'NHS Number': 'avsgvb',},
							{'ChildId':'id5',
							'NHS Number': 'varwvw',},
							])

child_info['Birthday_dt'] = pd.to_datetime(child_info['Birthday'], dayfirst=True, errors='coerce')

#lambda function - this was complex as HELL 
child_info['Age'] = child_info['Birthday_dt'].apply(lambda row: relativedelta(pd.to_datetime('today'), row).years)

child_info

In [14]:
# pip install openpyxl
import openpyxl

filepath = r"https://github.com/data-to-insight/ERN-sessions/raw/main/data/small%20excel.xlsx"

# with an excel, if oyu're reading multiple sheets, they come stored in a dictionary
# with sheet names as keys and dfs as values
df_dict = pd.read_excel(filepath, sheet_name=None)

print(df_dict.keys())

df_dict['Child Data']

dict_keys(['Child Data', 'Child extras', 'Sheet3'])


,ChildID,Sex,DOB,EHCP issued
0,id1,M,2014-12-05,N
1,id6,m,2016-01-06,n
2,id4,m,2013-11-09,Y
3,id3,F,2022-12-24,Y
4,id5,F,2020-01-15,NaN
5,id7,f,2022-09-17,y


In [ ]:
print(child_info.columns)

print(nhs_numbers.columns)

# if the columns names are the same for both tables you can simply use on='ChildID' rather than specifying a left and right on
# different types of merge, left, right, inner, outer
child_info = pd.merge(child_info, nhs_numbers, left_on='ChildId', right_on='ChildId', how='left')

child_info

In [ ]:
#identifiers = pd.merge(identifiers, characteristics, left_on='LAchildID', right_on='LAchildID', how='left')
identifiers = identifiers.merge(characteristics, on=['LAchildID', 'Unnamed: 0'], how='left')

identifiers

In [16]:
df = df_dict['Child Data' ].merge(df_dict['Child extras'], how='left', on='ChildID')
df = df.merge(df_dict['Sheet3'], how='left', on='Assessment Code' )

df


,ChildID,Sex,DOB,EHCP issued,Assessment Code,Assessment Outcome
0,id1,M,2014-12-05,N,1,Y
1,id6,m,2016-01-06,n,5,y
2,id4,m,2013-11-09,Y,3,N
3,id3,F,2022-12-24,Y,4,n
4,id5,F,2020-01-15,NaN,2,N
5,id7,f,2022-09-17,y,6,N
